evolution-strategies

Copyright (c) 2019 Patrick Deubel

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

evolution-strategies includes:

evolution-strategies-starter
Copyright (c) 2016 OpenAI (http://openai.com)

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

# Evaluation and Visualization of evolution-strategies

This Notebook can be used to easily evaluate and visualize results created with the `evolution-strategies.ipynb` Notebook.

When the functions are used, the explanations are only the basics. Feel free to look into the files in the `utils/` folder where the methods are implemented and documented.

## TrainingRun and Experiment

There are two objects that are mainly used to evaluate the results. An object of type `TrainingRun` describes one execution of the `run_master()` function of the main Notebook. It is therefore one result from the algorithm. `Experiment` objects therefore combine multiple `TrainingRun` objects which have the same values for the configuration. Meaning they have identical `config`, `model_structure` and `optimizations` objects.

In [ ]:
import os

from utils.es_utils import index_training_folder, index_experiments
from utils.config_values import LogColumnHeaders, EvaluationColumnHeaders

from IPython.display import display
from ipywidgets import Video

## Evaluation and Visualization of a TrainingRun

### Creating a TrainingRun object

For the creation of a TrainingRun object one has to pass the folder which was created during the TrainingRun to the `index_training_folder()` function. This will validate all objects in the folder and checks if indeed a valid TrainingRun has been found. If so it is returned.

In [ ]:
training_run = index_training_folder("training_runs/path-to-folder")

### Visualizing the TrainingRun

If you want to visualize the TrainingRun, you can specify the generation which shall be visualized. Note that for that to work, the corresponding model file must be present, otherwise the trained model can not be loaded and therefore no episode can be started. If you omit the generation, or use `-1` the last generation with a model file is used.

In [ ]:
video_file = training_run.visualize(env_seed=None, generation=-1, force=False)

In [ ]:
video = Video.from_file(video_file)
display(video)

### Evaluating the TrainingRun

Since only random evaluations take place during the actual training, the model files should be saved and the evaluation is then done afterwards. This way there are the same amount of evaluations in every generation.

In [ ]:
evaluation = training_run.evaluate(env_seed=None, num_evaluations=5, num_workers=os.cpu_count(), force=False, save=True)

In [ ]:
evaluation

### Plotting a TrainingRun

If you want to plot the reward against the timesteps for example the `plot_training_run` method is useful. It will take the Enums from `LogColumnHeaders` and `EvaluationColumnHeaders` as input to provide a way to uniformely access the data. TODO

## Evaluation and Visualization of an Experiment